# 👋 Getting started 2: Training adversarially robust 1-Lipschitz neural networks for classification

The goal of this series of tutorials is to show the different usages of `deel-lip`.

In this first notebook, we have shown how to create 1-Lipschitz neural networks with `deel-lip`.    
In this second notebook, we will show how to train adversarially robust 1-Lipschitz neural networks with `deel-lip`.    

In the course of this notebook, we will cover the following: 
1. [📚 Theoretical background](#theoretical_background)    
A brief theoretical background on adversarial robustness. This section can be safely skipped if one is not interested in the theory.
2. [💪 Training adversarially robust 1-Lipschitz neural networks on the MNIST dataset](#deel_keras)       
Using the MNIST dataset, we will show examples of training adversarially robust 1-Lipschitz neural networks using `deel-lip` loss functions `TauCategoricalCrossentropy` and `MulticlassHKR`.

We will also see that:
- the `MulticlassKR` loss function can be used to assess the adversarial robustness of the resulting models
- when training robust models, there is an accuracy-robustness trade-off.



## 📚 Theoretical background <a id='theoretical_background'></a>
### Adversarial attacks
In the context of classification problems, an adversarial attack is the result of adding an *adversarial perturbation* $\epsilon$ to the input data point $x$ of a trained predictive model $A$, with the intent to change its prediction.

In simple mathematical terms, an adversarial example (i.e. a succesful adversarial attack) can be transcribed as below:

$$A(x)=y_1,$$
$$A(x+\epsilon)=y_{\epsilon},$$
where:
$$y_1\neq y_\epsilon.$$

### An adversarial example

The following example is directly taken from https://adversarial-ml-tutorial.org/introduction/.

![pigs.png](../assets/pigs.png)

The first image is correctly classified as a **pig** by a classifier. The second image is incorrectly classified as an **airplane** by the same classifier. 

While both images cannot be distinguished from our (human) perspective, the second image is in fact the result of surimposing "noise" (i.e. adding an adversarial perturbation) to the original first image.

Below is a visualization of the added noise, zoomed-in by a factor of 50 so that we can see it:
![noise.png](../assets/noise.PNG)

### Adversarial robustness of 1-Lipschitz neural network
The adversarial robustness of a predictive model is its ability to remain accurate and reliable when subjected to adversarial perturbations.  

Practically, training an adversarial robust predictive model consists in obtaining a formal guarantee that it remains accurate, even when exposed against adversarial attacks that are defined within certain specified constraints.

A major advantage of 1-Lipschitz neural networks is that they can offer provable guarantees on their robustness for any particular input $x$.

One can formulate such a guarantee, in terms such as follow:

> "For an input $x$, we can certify that are no adversarial perturbations constrained to be under the certificate $\epsilon_x$ that will change our model's prediction."

In simple mathematical terms:  

For a given $x$, $\forall \epsilon$ such that $||\epsilon||<\epsilon_x$, we obtain that:
$$A(x)=y,$$
$$A(x+\epsilon)=y_{\epsilon},$$
then:
$$y_{\epsilon}=y.$$

💡 The *certificate* $\epsilon_x$ attached to an input $x$ can be deduced from the logits of the 1-Lipschitz neural network used. This will be the topic of another notebook. In this notebook, robustness will be approximated by the `MulticlassKR` loss, as shown later.

💡 Depending on the type of norm you choose (e.g. L1 or L2), the guarantee you can offer will differ, as $||\epsilon||_{L2}<\epsilon_x$ and $||\epsilon||_{L1}<\epsilon_x$ are not equivalent.

As such, other examples of guarantees with a more precise formulation would be:
> "For an input $x$, we can certify that are no adversarial perturbations constrained to be within a $\text{L2}$-norm ball of certificate $\epsilon_{x,\text{L2}}$ that will change our model's prediction."

> "For an input $x$, we can certify that are no adversarial perturbations constrained to be within a $\text{L1}$-norm ball of certificate $\epsilon_{x,\text{L1}}$ that will change our model's prediction."



## 💪 Training adversarially robust 1-Lipschitz neural networks on the MNIST dataset <a id='deel_keras'></a>


### 💾 MNIST dataset
MNIST dataset contains a large number of 28x28 digit images to which are associated digit labels.

In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np

In [4]:
# Load MNIST Database
(X_train, y_train_ord), (X_test, y_test_ord) = mnist.load_data()

# standardize and reshape the data
X_train = np.expand_dims(X_train, -1) / 255
X_test = np.expand_dims(X_test, -1) / 255

# one hot encode the labels
y_train = to_categorical(y_train_ord)
y_test = to_categorical(y_test_ord)

### 🔮 Prediction Model

We show two cases. In the first case, we use `deel-lip`'s `TauCategoricalCrossentropy` from the `losses` submodule. In the second case, we use this other loss function from `deel-lip`: `MulticlassHKR`.

In particular, we will show how these functions can be parametrized to increase the robustness of our predictive models. We will also see that generally, there is a compromise between the robustness and the accuracy of our models (i.e. better robustness generally comes at the price of a decrease in performance).

Because we will need to instanciate four times the same model in our examples, we encapsulate the code to create our model in a function for conciseness:

In [1]:
import deel
from deel import lip

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Flatten


activation=lip.activations.GroupSort2

def create_conv_model(name_model):
    """
    A simple convolutional neural network, made to be 1-Lipschitz.
    """
    model= lip.Sequential(
        [
        Input(shape=X_train.shape[1:]),
        
        lip.layers.SpectralConv2D(
                filters=16,
                kernel_size=(3, 3),
                activation=activation(),
                use_bias=True,
                kernel_initializer="orthogonal",
            ),
        
        lip.layers.ScaledL2NormPooling2D(pool_size=(2, 2), data_format="channels_last"),
            
        lip.layers.SpectralConv2D(
                filters=32,
                kernel_size=(3, 3),
                activation=activation(),
                use_bias=True,
                kernel_initializer="orthogonal",
            ),
        
        lip.layers.ScaledL2NormPooling2D(pool_size=(2, 2), data_format="channels_last"),
        
        Flatten(),
        
        lip.layers.SpectralDense(
                64,
                activation=activation(),
                use_bias=True,
                kernel_initializer="orthogonal",
            ),
        
        lip.layers.SpectralDense(
                y_train.shape[-1], activation=None, use_bias=False, kernel_initializer="orthogonal"
            ),
        ],
    
        k_coef_lip=1.0,
        name=name_model,
    )

    return model

#### Cross-entropy loss: `TauCategoricalCrossentropy`

Similar to the classes we have seen in "Getting started 1", the `TauCategoricalCrossentropy` class is similar to its equivalent in `keras`, but it comes with a settable temperature parameter `tau`. This parameter will allow to adjust the robustness of our model. The lower the temperature is, the more robust our model becomes, but it also becomes less accurate.

To show the impact of the `tau` on both the performance and robustness of our model, we will train two models on the MNIST dataset. The first model will have a temperature of 100, the second model will have a temperature of 3.

In [5]:
# high-temperature model
model_1 = create_conv_model("cross_entropy_model_1")

temperature_1=100.

model_1.compile(
    loss=lip.losses.TauCategoricalCrossentropy(tau=temperature_1),
    optimizer=Adam(1e-4),
    # notice the use of lip.losses.MulticlassKR(), to assess adversarial robustness
    metrics=["accuracy", lip.losses.MulticlassKR()],
)

In [7]:
# low-temperature model
model_2 = create_conv_model("cross_entropy_model_2")

temperature_2=3.

model_2.compile(
    loss=lip.losses.TauCategoricalCrossentropy(tau=temperature_2),
    optimizer=Adam(1e-4),
    metrics=["accuracy", lip.losses.MulticlassKR()],
)

💡 Notice that we use the accuracy metric to measure the performance, and we use the `MulticlassKR` loss to measure adversarial robustness. The latter acts as a proxy of our model's average certificates, **: the higher this measure is, the more robust our model is**.   

**🚨 Note:** *This is true only for 1-Lipschitz neural networks*

We fit both our models and observe the results.

In [ ]:
# fit the high-temperature model
result_1=model_1.fit(
    X_train,
    y_train,
    batch_size=256,
    epochs=2,
    validation_data=(X_test, y_test),
    shuffle=True,
    #verbose=1,
)

In [9]:
# fit the low-temperature model
result_2=model_2.fit(
    X_train,
    y_train,
    batch_size=256,
    epochs=2,
    validation_data=(X_test, y_test),
    shuffle=True,
    #verbose=1,
)

In [42]:
# metrics for the high-temperature model => performance-oriented 
print(f"Model accuracy: {result_1.history['val_accuracy'][-1]:.4f}")
print(f"Model MulticlassKR: {result_1.history['val_MulticlassKR'][-1]:.4f}")
print(f"Loss' temperature: {model_1.loss.tau.numpy():.1f}")

Model accuracy: 0.9514
Model MulticlassKR: 0.1277
Loss' temperature: 100.0


In [43]:
# metrics for the low-temperature model => robustness-oriented
print(f"Model accuracy: {result_2.history['val_accuracy'][-1]:.4f}")
print(f"Model MulticlassKR: {result_2.history['val_MulticlassKR'][-1]:.4f}")
print(f"Loss' temperature: {model_2.loss.tau.numpy():.1f}")

Model accuracy: 0.9276
Model MulticlassKR: 1.7535
Loss' temperature: 3.0


When decreasing the temperature, we observe an increase in robustness, but a decrease in accuracy.

#### Hinge-Kantorovich–Rubinstein loss: `MulticlassHKR`

We work in the same way as in the previous section. The difference lies in the parameters that control the robustness.

We count two of them: `min_margin` (minimal margin) and `alpha` (regularization factor).

As will be shown in the following, a higher minimal margin and a lower alpha increases robustness. 

In [28]:
# performance-oriented model
model_3 = create_conv_model("HKR_model_3")

min_margin_3=1
alpha_3=30

model_3.compile(
    loss=lip.losses.MulticlassHKR(min_margin=min_margin_3,alpha=alpha_3),
    optimizer=Adam(1e-4),
    metrics=["accuracy", lip.losses.MulticlassKR()],
)

In [39]:
# robustness-oriented model
model_4 = create_conv_model("HKR_model_4")

min_margin_4=3
alpha_4=10

model_4.compile(
    loss=lip.losses.MulticlassHKR(min_margin=min_margin_4,alpha=alpha_4),
    optimizer=Adam(1e-4),
    metrics=["accuracy", lip.losses.MulticlassKR()],
)

We fit both our models and observe the results.

In [30]:
# fit the model
result_3=model_3.fit(
    X_train,
    y_train,
    batch_size=256,
    epochs=2,
    validation_data=(X_test, y_test),
    shuffle=True,
    verbose=1,
)

Epoch 1/2
235/235 [==============================] - 26s 96ms/step - loss: 10.3240 - accuracy: 0.7246 - MulticlassKR: 0.8375 - val_loss: 3.9340 - val_accuracy: 0.8969 - val_MulticlassKR: 1.3823
Epoch 2/2
235/235 [==============================] - 22s 92ms/step - loss: 2.9818 - accuracy: 0.9019 - MulticlassKR: 1.5695 - val_loss: 1.9182 - val_accuracy: 0.9244 - val_MulticlassKR: 1.7646


In [40]:
# fit the model
result_4=model_4.fit(
    X_train,
    y_train,
    batch_size=256,
    epochs=2,
    validation_data=(X_test, y_test),
    shuffle=True,
    verbose=1,
)

Epoch 1/2
235/235 [==============================] - 25s 95ms/step - loss: 16.5619 - accuracy: 0.5336 - MulticlassKR: 1.1116 - val_loss: 8.8919 - val_accuracy: 0.7831 - val_MulticlassKR: 2.0187
Epoch 2/2
235/235 [==============================] - 21s 91ms/step - loss: 7.6997 - accuracy: 0.7989 - MulticlassKR: 2.2113 - val_loss: 6.5351 - val_accuracy: 0.8436 - val_MulticlassKR: 2.4005


In [34]:
# performance-oriented model
print(f"Model accuracy: {result_3.history['val_accuracy'][-1]:.4f}")
print(f"Model MulticlassKR: {result_3.history['val_MulticlassKR'][-1]:.4f}")
print(f"Loss' minimum margin: {model_3.loss.min_margin.numpy():.1f}")
print(f"Loss' alpha: {model_3.loss.alpha.numpy():.1f}")

Model accuracy: 0.9244
Model MulticlassKR: 1.7646
Loss' minimum margin: 1.0
Loss' alpha: 30.0


In [41]:
# robustness-oriented model
print(f"Model accuracy: {result_4.history['val_accuracy'][-1]:.4f}")
print(f"Model MulticlassKR: {result_4.history['val_MulticlassKR'][-1]:.4f}")
print(f"Loss' minimum margin: {model_4.loss.min_margin.numpy():.1f}")
print(f"Loss' alpha: {model_4.loss.alpha.numpy():.1f}")

Model accuracy: 0.8436
Model MulticlassKR: 2.4005
Loss' minimum margin: 3.0
Loss' alpha: 10.0


We confirmed experimentally the accuracy-robustness trade-off: a higher minimal margin and a lower alpha increases robustness, but also decreases accuracy.

## 🎉 Congratulations
You now know how to train adversarially robust 1-Lipschitz neural networks!

👓 Interested readers can learn more about the role of loss functions and the accuracy-robustness trade-off which occurs when training adversarially robust 1-Lipschitz neural network in the following paper:   
 [Pay attention to your loss: understanding misconceptions about 1-Lipschitz neural networks](https://arxiv.org/abs/2104.05097).